### This script applies fuzzy matching from seat geek, outputs the matches and then employs some categorical variable selection via logistic regression to prune the dummy feature set to only include the features deemed relevant via having a coefficent > .0005 which results in a slimmed down feature set of 153 features.
#### Steps:
#### -Take employee title data from KagDataCleansing and run through a preprocess script I wrote located in the data library, which makes the text all lowercase, removes non-ascii text, handles some popular abbreviations, and removes punctuation.
#### -Use text matching script from https://github.com/seatgeek/fuzzywuzzy
#### -Parameters used were setting a threshold of .74 and after testing various methods, fuzz_token_sort_ratio was chosen as the matching parameter of choice
#### -Pipe output into L2 logistic regression, while only keeping titles that have <br> coeficients >.0005 which reduces the unique title count to 153 features.
#### Output:
#### -f_norm_emp_title - removal of punctuation, non-ascii characters, lowercase
#### -fw_emp_title - output of fuzzymatch
#### - fw_emp_title_gt0005 - slimmed down titles only showing some degree of importance

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from time import time
import string
#normalize data crossreference match what was put into fuzzywuzzy

In [2]:
cd Data

/Users/Shalu/Dropbox/Coursework/DS/KLC/Submission/Data


In [4]:
dfc = pd.read_pickle('df_emp_1020.p')

In [5]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232487 entries, 0 to 235628
Data columns (total 28 columns):
loan_amnt                     232487 non-null float64
term                          232487 non-null object
int_rate                      232487 non-null float64
grade                         232487 non-null object
sub_grade                     232487 non-null object
emp_title                     219418 non-null object
emp_length                    232487 non-null int64
home_ownership                232487 non-null object
annual_inc                    232487 non-null float64
verification_status           232487 non-null object
loan_status                   232487 non-null object
purpose                       232487 non-null object
zip_code                      232487 non-null object
addr_state                    232487 non-null object
dti                           232487 non-null float64
delinq_2yrs                   232487 non-null float64
earliest_cr_line              232487 

## Quickly inspect popular abbreviations to be addressed in preprocessing script

In [114]:
dfc['LCase_Count'] = dfc['LCase'].str.len()

In [121]:
dfc['LCase'][(dfc.LCase_Count<5)].value_counts(dropna=False).head(30)

rn      2019
ceo      354
lpn      339
cna      299
cook     236
tech     219
chef     203
vp       177
cfo      162
csr      151
gm       101
it       100
coo       92
lvn       83
emt       67
cpa       61
lead      60
avp       50
svp       49
mgr       39
e-6       39
cto       38
csa       37
rn        36
cio       34
hha       34
crna      32
hr        32
e-7       31
pca       25
Name: LCase, dtype: int64

## Pre_Process Columns

In [5]:
print dfc.shape
print dfc.emp_title.unique().shape

(232487, 28)
(74553L,)


In [5]:
dfc.columns

Index([u'loan_amnt', u'term', u'int_rate', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'loan_status', u'purpose', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util',
       u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc',
       u'loan_label', u'emp_title_flag', u'fico_avg'],
      dtype='object')

In [6]:
import preprocess_emp_title as preproc
dfc = preproc.preprocess(dfc)

In [7]:
print dfc.shape
print dfc.f_norm_emp_title.unique().shape

(232487, 29)
(60067L,)


In [8]:
dfc = dfc.sort_values(by='f_norm_emp_title')

In [9]:
dfc.reset_index(inplace=True)

## Use FuzzyWuzzy Algorithm to Extract Like Matches

In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [14]:
emp_title = dfc.f_norm_emp_title.value_counts().index.tolist()

In [15]:
#93/200
#5000 - 591 seconds | 9 minutes
#10000 - 2275 seconds
#All 76148 seconds/21 hours
#85/500
#All 10042 seconds/2.85 hours on PC

#import caffeine

threshold = 74
limit_f = 10000000

start = time()
d={}
for i in emp_title:
    #score = process.extract(i, emp_title, limit=limit_f)
    score = process.extractBests(i, emp_title, limit=limit_f, scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
    d[i]=[]
    for j,k in score:
        d[i].append(j)
        emp_title.remove(j)
end = time()
print "{:.4f} seconds".format((end - start))
#caffeine.off()

11438.1320 seconds


In [16]:
#.1086 at 100
df_d = pd.DataFrame.from_dict(d, orient="index")

In [17]:
df_d.to_pickle('df_d_fuzzy_SUCCESS_74_Best_token_sort.p')

In [18]:
emp_mat = df_d.as_matrix()

In [19]:
emp_mat.shape

(8675L, 400L)

In [20]:
def rem_null (mat):
    c= []
    for i in mat:
        clean = [x for x in i if str(x) != 'None']
        c.append(clean)
    return c

In [31]:
#emp_mat_c = rem_null(emp_mat)
11438/3600

3

In [22]:
def find_unique(mat):
    c=[]
    for i in mat:
        c.append(np.unique(i))
    return c

In [23]:
emp_mat_d = find_unique(emp_mat) #emp_mat_c

In [24]:
#remove Null
emp_mat_e = rem_null(emp_mat_d)

In [25]:
emp_mat_e

[['lead xray tech',
  'therapy tech',
  'xr tech',
  'xraty tech',
  'xray ct tech',
  'xray tech',
  'xray tech iii',
  'xray technician',
  'xrayct tech'],
 ['student relations clerk'],
 ['charness'],
 ['district clerksec to superintendent'],
 ['foreman union painter', 'union painter', 'union painterplasterer'],
 ['contract',
  'contract coord',
  'contract oversight',
  'contract pilot',
  'contract writing',
  'contracting',
  'contractor',
  'contractor sales',
  'courier contractor',
  'driver contractor',
  'fe to m6 contractor',
  'fedex contractor',
  'hvac contractor',
  'independant contractor',
  'independent  contractor',
  'independent contractor',
  'indipendant contractor',
  'it contractor',
  'painting contractor',
  'plumbing contractor',
  'sap contractor',
  'scientific contractor',
  'self contractor',
  'set constructor',
  'software contractor',
  'sub contractor',
  'subcontractor',
  'tile contractor'],
 ['2nd pressman', 'asst pressman', 'head pressman', 'lead

In [35]:
dfc.head(5)

,index,loan_amnt,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,pub_rec,revol_bal,revol_util,initial_list_status,collections_12_mths_ex_med,mort_acc,loan_label,emp_title_flag,fico_avg,f_norm_emp_title
0,138828,35000.0,60 months,14.49,C,C4,0-3,10,MORTGAGE,120000.0,...,0.0,29782.0,54.1,w,0.0,1.0,0,1,737.0,03
1,196305,25000.0,36 months,15.61,C,C5,0-3,5,MORTGAGE,88800.0,...,0.0,19647.0,61.0,f,0.0,2.0,1,1,722.0,03
2,123573,17350.0,36 months,16.99,D,D3,04 LCDR,10,MORTGAGE,100000.0,...,0.0,29408.0,73.9,w,0.0,10.0,0,1,672.0,04 lcdr
3,63639,21600.0,60 months,14.99,C,C5,06 Journymen,1,MORTGAGE,61000.0,...,0.0,27820.0,53.8,w,0.0,6.0,0,1,677.0,06 journymen
4,50745,8000.0,36 months,10.99,B,B3,0perator,7,MORTGAGE,100000.0,...,0.0,13292.0,24.8,f,0.0,1.0,0,1,702.0,0perator


In [26]:
#dfc_pos = tmp_dfc_pos[::]
def map_matches(x, mat):
    for i in range(len(mat)):
        for j in mat[i]:
            if str(j)==x:
                return mat[i][0]
        i += 1

In [27]:
dfc.columns

Index([u'index', u'loan_amnt', u'term', u'int_rate', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'loan_status', u'purpose', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util',
       u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc',
       u'loan_label', u'emp_title_flag', u'fico_avg', u'f_norm_emp_title'],
      dtype='object')

In [28]:
#8893 seconds ==> 148 minutes
#3528 seconds ==>  58 minutes
start = time()
dfc['fw_emp_title'] = dfc.f_norm_emp_title.apply(lambda x: map_matches(x, emp_mat_e))
end = time()
print "{:.4f} seconds".format((end - start))


3633.4880 seconds


In [29]:
#22655 Uniques After Matching at .93
#4943 Uniques After Matching at .85
#8676 After at .74 and Token Sort Ratio
print "%d Unique Titles before matching" % len(np.unique(dfc.f_norm_emp_title))
print "%d Unique Titles after matching" % len(np.unique(dfc.fw_emp_title))

60067 Unique Titles before matching
8676 Unique Titles after matching


In [30]:
#dfc.to_pickle('delemp500_74.p')

In [32]:
dfc[dfc.fw_emp_title.isnull()]

,index,loan_amnt,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,revol_bal,revol_util,initial_list_status,collections_12_mths_ex_med,mort_acc,loan_label,emp_title_flag,fico_avg,f_norm_emp_title,fw_emp_title
0,138828,35000.0,60 months,14.49,C,C4,0-3,10,MORTGAGE,120000.00,...,29782.0,54.1,w,0.0,1.0,0,1,737.0,03,None
1,196305,25000.0,36 months,15.61,C,C5,0-3,5,MORTGAGE,88800.00,...,19647.0,61.0,f,0.0,2.0,1,1,722.0,03,None
2,123573,17350.0,36 months,16.99,D,D3,04 LCDR,10,MORTGAGE,100000.00,...,29408.0,73.9,w,0.0,10.0,0,1,672.0,04 lcdr,None
9,160949,10000.0,36 months,11.67,B,B4,1 unracker lead. 2 Direct care.763-,10,RENT,55000.00,...,6438.0,28.2,f,0.0,0.0,0,1,707.0,1 unracker lead 2 direct care763,None
10,139410,12700.0,60 months,19.52,E,E2,10,1,OWN,30240.00,...,18132.0,55.6,w,0.0,0.0,0,1,702.0,10,None
11,212238,21000.0,36 months,18.25,D,D3,10,6,MORTGAGE,104000.00,...,12794.0,69.2,f,0.0,2.0,0,1,682.0,10,None
13,66303,20000.0,60 months,13.98,C,C3,100% Owner,10,MORTGAGE,60822.00,...,22064.0,90.8,f,0.0,3.0,0,1,687.0,100 owner,None
14,222766,30000.0,60 months,18.25,D,D3,105000.00,1,MORTGAGE,105000.00,...,37147.0,47.9,f,0.0,4.0,0,1,682.0,10500000,None
17,146275,8000.0,36 months,10.99,B,B3,1170 Carew st,3,OWN,45000.00,...,4981.0,94.0,f,0.0,0.0,0,1,687.0,1170 carew st,None
20,140480,10000.0,60 months,22.15,E,E5,121000,2,MORTGAGE,160000.00,...,58032.0,81.1,f,0.0,6.0,0,1,667.0,121000,None


In [37]:
#below is list of occupations that were entered but didn't have a suitable match
dfc.fw_emp_title.value_counts(dropna=False).head(10)

na                     13110
NoMatch                11405
3d manager              7870
nure                    6443
ag teacher              6024
area amanager           5090
911 supervisor          3703
aco project manager     3563
oner                    2490
cales                   2368
Name: fw_emp_title, dtype: int64

In [33]:
#Reassign NaN to No Match
print len(dfc[dfc.fw_emp_title.isnull()])
dfc['fw_emp_title'][dfc.fw_emp_title.isnull()] = 'NoMatch'
print len(dfc[dfc.fw_emp_title.isnull()])

11405
0


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [34]:
print len(dfc[dfc['fw_emp_title']=='NoMatch'])

11405


In [50]:
dfc[['emp_title','f_norm_emp_title','fw_emp_title']][dfc['fw_emp_title']=='na'].head(5)

,emp_title,f_norm_emp_title,fw_emp_title


In [48]:
print len(dfc[dfc['fw_emp_title']=='na'])
dfc['fw_emp_title'][dfc['fw_emp_title']=='na'] = 'empty'
print len(dfc[dfc['fw_emp_title']=='empty'])

13110
13110


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [36]:
print dfc.columns
dfc.shape

Index([u'index', u'loan_amnt', u'term', u'int_rate', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'loan_status', u'purpose', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util',
       u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc',
       u'loan_label', u'emp_title_flag', u'fico_avg', u'f_norm_emp_title',
       u'fw_emp_title'],
      dtype='object')


(232487, 31)

In [49]:
len(np.unique(dfc.fw_emp_title))

8676

In [141]:
### Top 200 Unique Matched Names Code Not Used
#  (dfc['fw_emp_title'][dfc.loan_label==1].value_counts(dropna=False)).iloc[0:200]

# emp200 = (dfc['fw_emp_title'][dfc.loan_label==1].value_counts(dropna=False)).iloc[0:200].index

# def map_top_matches(x, mat):
#         if x in mat:
#             return x
#         else:
#             return "NoMatch"

# start = time()
# dfc['top200_fw_emp_title'] = dfc.fw_emp_title.apply(lambda x: map_top_matches(x, emp200))
# end = time()
# print "{:.4f} seconds".format((end - start))

## Logistic Regression For Variable Importance

In [5]:
#df_reduce = dfc.copy()
df_reduce['emp_length_cat'] = df_reduce.emp_length.astype('category')
df_reduce.fillna(0.0,inplace=True)
df_reduce.fillna(0,inplace=True)

In [5]:
random_state=401
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import scale
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

def my_scorer(clf, X, y):
    y_pred_prob = clf.predict_proba(X)[:,1]
    return roc_auc_score(y, y_pred_prob)

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=random_state)
scoring_function = my_scorer

In [6]:
logreg = LogisticRegression(penalty='l2', C=.00001, random_state=random_state)

In [11]:
features = ['fw_emp_title']

target = ['loan_label']

X=df_reduce[features]
print X.columns
X = pd.get_dummies(X)
col = X.columns
print X.shape
#X = scale(X)
y=df_reduce[target].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify=y, test_size=.25, random_state=random_state)

Index([u'fw_emp_title'], dtype='object')
(232487, 8676)


In [12]:
import caffeine

start = time()
logreg.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)


62.7717 seconds


In [13]:
y_pred_prob = logreg.predict_proba(X_test)[:,1]

print roc_auc_score(y_test, y_pred_prob)
caffeine.off()

0.492611747512


In [103]:
t = pd.DataFrame(zip(col, logreg.coef_[0]), columns=['features','coef'])
t['coef']=abs(t.coef)
t.sort_values(by='coef', ascending=False).head(15)

,features,coef
90,fw_emp_title_NoMatch,0.023121
4168,fw_emp_title_empty,0.021995
56,fw_emp_title_3d manager,0.014485
6449,fw_emp_title_nure,0.012993
531,fw_emp_title_ag teacher,0.012509
844,fw_emp_title_area amanager,0.009985
259,fw_emp_title_aco project manager,0.007633
85,fw_emp_title_911 supervisor,0.007087
876,fw_emp_title_area president,0.005097
6518,fw_emp_title_oner,0.004719


In [104]:
len('fw_emp_title_')

13

In [105]:
#remove dummy column string concatenated and 
t['features_red'] = t.features.apply(lambda x: x[13:])

### Take coefficients greater than .0005 and strip the rest. Leaving 153 features for Employee Title

In [107]:
emp001 = t[t.coef>0.0005]
print len(emp001)

153


In [177]:
def map_top_matches(x, mat):
        if any(x == (emp001.features_red)):
            return x
        else:
            return "NoMatch"

In [178]:
start=time()
df_reduce['fw_emp_title_gt0005'] = df_reduce.fw_emp_title.apply(lambda x: map_top_matches(x, str(emp001.features_red)))
end = time()
print "{:.4f} seconds".format((end - start))

570.0527 seconds


In [182]:
#153 unique titles
len(np.unique(df_reduce.fw_emp_title_gt0005))

153

In [184]:
df_reduce.shape

(232487, 33)

In [7]:
df_reduce.columns

Index([u'index', u'loan_amnt', u'term', u'int_rate', u'grade', u'sub_grade',
       u'emp_title', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'loan_status', u'purpose', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util',
       u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc',
       u'loan_label', u'emp_title_flag', u'fico_avg', u'f_norm_emp_title',
       u'fw_emp_title', u'emp_length_cat', u'fw_emp_title_gt0005'],
      dtype='object')

In [3]:

#df_reduce.to_pickle('dfc_fw_emp_1027_74.p')